## Homework 7

Carlos Lizama  
N17866309  

### Brief description of the homework
Consider the *equilibrium price function* 

$$ q(x,y) = \beta \sum_{y \in S} [d(y) + p(y)] q(x,y) $$


In this homework I first provide codes to solve for the equilibrium price function with an iterative procedure based on the Banach contraction mapping theorem. The code is tested for different specifications given in the home. Then the model is extendend to account for heterogeneous beliefs and I show conditions such that the mapping defined by the equilibrium price function for the model with heterogeneous beliefs is a uniform contraction, so the algorithm of iteratively applying the mapping converges to the unique solution. Finally, I solve the problem with heterogeneous beliefs.

### Exercise 1:
In this exercise I compute the equilibrium price function by applying the map $T p(x) = \beta \sum_{y \in S} [d(y) + p(y)] q(x,y)$ iteratively. This algorithm converges to the unique solution because of the contraction mapping theorem. The next function takes the primities of the problem and returns the equilibrium price. 

In [1]:
"""
This code solves for the equilibrium price function as in Lucas (1978). The parameters are the discount factor beta,
the dividends vector d, and the transition matrix M.
"""
function solve_eq_price_function(beta, d, M, tol=1e-4)
    S = length(d)
    p = ones(S)
    distance = 1
    
    while distance >= tol
        # define map T
        Tp = beta*M*(d + p)
        distance = norm(p-Tp,Inf)
        p = copy(Tp)
    end
    
    return p
end

solve_eq_price_function (generic function with 2 methods)

Now to test the code I compute the equilibrium price for an economy with the following paramteres

In [2]:
beta = .98
d = [1, 0, 0]
q1 = [.971 .029 0; .145 .778 .077; 0 .508 .492]
p_eq1 = solve_eq_price_function(beta, d, q1)

3-element Array{Float64,1}:
 40.7549
 35.956 
 34.5672

Now I compute the equilibrium price with a different matrix.

In [3]:
q2 = [.871 0.029 .1; .145 .778 .077; 0 .508 .492]

3x3 Array{Float64,2}:
 0.871  0.029  0.1  
 0.145  0.778  0.077
 0.0    0.508  0.492

Note that the only difference between the matrices is that the last matrix has lower probability of staying in the normal growth state conditional on being in that state and it has higher probability of transit from the normal growth state to the severe recession state. More precisely, a mass of 0.1 has been taken away from the probabilty of staying in the NG state and it has been put to the probabilty of moving to a severe recession. The new equilibrium price is

In [4]:
p_eq2 = solve_eq_price_function(beta, d, q2)

3-element Array{Float64,1}:
 23.4963
 21.094 
 20.2791

The equilibrium price is lower for every state of the economy. The intuition of this result is straightforward. The change in the transition matrix makes that the economy spend much more time in states in which no dividends are paid (mild and sereve recession). Therefore, the price of the asset must be lower. To put it differently, the value of the asset in the normal growth state is lower because now it is expected that the asset would paid for fewer periods, the probability of leaving the state is higher. In the mild and severe recession states, the asset is also less valuable. Even though the transition probabilty of reaching the normal growth state are the same, the value of the asset in that state is lower, and hence so are the values for the asset in the other states.

### Exercise 2: 
In this exercise I show that the mappping defined by $$ Tp(x) = \beta \max_{i \in \{1,2\}} \left\{ \sum_{y \in S} [d(y) + p(y)] q_i(x,y) \right\} $$ is a uniform contraction.

To do that, I need to prove that $T$ is a map $b \mathbb{R}^S$ to itself and then show that it is a uniform contraction. The condition required to these proofs to work is that $||d||_{\infty}$ is bounded.

#### Proof of that $T$ is a map from $b \mathbb{R}^S$ to itself.
This is to proof that given a bounded function $p$ then $T(p)$ is a bounded function as well. In fact
\begin{align}
|p(x)| &= \beta \left| \max_{i \in \{1,2\}} \left\{ \sum_{y \in S} [d(y)+p(y)] q_i(x,y) \right\} \right| \\
& \leq \beta \left| \sum_{i \in \{1,2\}} \sum_{y \in S} [d(y)+p(y)] q_i(x,y) \right| \\
& \leq \beta \sum_{i \in \{1,2\}} \sum_{y \in S} \left| d(y) + p(y) \right|q_i(x,y) \\
& \leq \beta \sum_{i \in \{1,2\}} \sum_{y \in S} (||d||_{\infty} + ||p||_{\infty})q_i(x,y) \\
& = 2 \beta (||d||_{\infty} + ||p||_{\infty})
\end{align}
and taking the supremum on the LHS gives $ || Tp ||_{\infty} < \infty$

Now let's prove that $T$ is in fact a uniform contraction.

#### Proof of that $T$ is a uniform contraction.
Want to show that the following condition holds $ ||Tp - Tp'||_{\infty} \leq \beta ||p-p'||_{\infty} $

For a given $x$, we have:
\begin{align}
|Tp(x) - Tp'(x)| &= \beta \left| \max_{i \in \{1,2\}} \left\{ \sum_{y \in S} [d(y)+p(y)]q_i(x,y) \right\} - \max_i \left\{ \sum_{y \in S} [d(y)+p'(y)]q_i(x,y) \right\} \right|  \\
& \leq \beta \max_{i \in \{1,2\}} \left| \sum_{y \in S} [p(y) - p'(y)]q_i(x,y) \right| \\
& \leq \beta \max_{i \in \{1,2\}} \sum_{y \in S} |p(y)-p(y')| q_i(x,y) \\
& \leq \beta \max_{i \in \{1,2\}} \sum_{y \in S} ||p-p'||_{\infty} q_i(x,y) \\
& = \beta ||p-p'||_{\infty}  \max_{i \in \{1,2\}} \sum_{y \in S} q_i(x,y) \\
& = \beta ||p-p'||_{\infty}
\end{align}

Therefore, since $x$ was arbitrary, we conclude that $||Tp-Tp'||_{\infty} \leq \beta ||p-p'||_{\infty}$
In the second line I use the hint, which I now prove.

Want to show: for any $\{a_i\}$ and $\{b_i\}$ the following inequality holds: $$ | \sup_i a_i - \sup_i b_i | \leq \sup_i |a_i - b_i| $$

Note that the last inequality is equivalent to the following two inequalities:
\begin{align}
- \sup_i |a_i - b_i| \leq \sup_i a_i - \sup b_i \leq \sup_i |a_i - b_i|
\end{align}

I prove both inequality separately. I will also use the following property: $$\sup_i (x_i + y_i) \leq \sup x_i + \sup_i y_i $$
which can be proved trivially since $x_i \leq \sup_i x_i$ and $y_i \leq \sup_i y_i$, then summing up $ (x_i + y_i) \leq \sup_i x_i + \sup_i y_i$ and taking $\sup$ in both sides we obtain $\sup_i (x_i + y_i) \leq \sup x_i + \sup_i y_i $

I proceed with the proof
\begin{align}
a_i &= (a_i - b_i) + b_i \\
&\leq |a_i - b_i| + b_i  \\
\Rightarrow \sup_i a_i &\leq \sup_i |a_i - b_i| + \sup_i b_i \\
\iff \sup_i a_i - \sup_i b_i &\leq \sup_i |a_i - b_i|    \qquad (*)
\end{align}

Doing the same trick with $b_i$
\begin{align}
b_i &= (b_i - a_i) + a_i \\
&\leq |b_i - a_i| + a_i \\ 
&= |a_i - b_i| + a_i  \\
\Rightarrow \sup_i b_i &\leq \sup_i |a_i - b_i| + \sup_i a_i \\
\iff \sup_i b_i - \sup_i a_i &\leq \sup_i |a_i - b_i|  \\
\iff  -\sup_i |a_i - b_i| & \leq \sup_i a_i - \sup_i b_i \qquad (* *)
\end{align}

Equations $(*)$ and $(* *)$ show the desired result.

Finally, I solve the model for the heterogeneous beliefs. The following function does that.

In [5]:
"""
This code solves for the equilibrium price function in an economy with heterogeneous beliefs. 
The parameters are the discount factor beta (common across agents), the dividends vector d, 
and the transition matrices M1 and M2.
"""
function eq_price_heterogeneous(beta, d, M1, M2, tol=1e-4)
    S = length(d)
    p = ones(S)
    distance = 1
    
    while distance>=tol
        # define map T for each matrix
        T1p = beta*M1*(d + p)
        T2p = beta*M2*(d + p)
        Tp = vec(maximum(hcat([T1p T2p]),2))
        distance = norm(p-Tp,Inf)
        p = copy(Tp)
    end
    
    return p
end

eq_price_heterogeneous (generic function with 2 methods)

The equilibrium price with matrices $q_1$ and $q_2$ is

In [6]:
p_eq3 = eq_price_heterogeneous(beta,d,q1,q2)

3-element Array{Float64,1}:
 40.7549
 35.956 
 34.5672